In [33]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
from pyspark.sql import SparkSession

In [35]:
spark=SparkSession.builder.appName("Session").getOrCreate()

In [36]:
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import os
import numpy as np
from datetime import datetime

In [37]:
%%capture
%pip install -r requirements.txt
!python -m nltk.downloader stopwords
!python -m nltk.downloader wordnet
!python -m nltk.downloader omw-1.4

In [38]:
binance_coins = ['BNB', 'XRP', 'BCHABC', 'LTC', 'LINK', 'ETC', 'TRX', 'EOS', 'XLM', 'ADA', 'BCH', 'QTUM', 'VET', 'EUR', 'BULL', 'BEAR', 'ETHBULL', 'ETHBEAR', 'ICX', 'BTS', 'BNT', 'ATOM', 'DASH', 'NEO', 'WAVES', 'XTZ', 'EOSBULL', 'EOSBEAR', 'XRPBULL', 'XRPBEAR', 'BAT', 'ENJ', 'NANO', 'ONT', 'RVN', 'STRAT', 'AION', 'ALGO', 'BTT', 'TOMO', 'XMR', 'ZEC', 'BNBBULL', 'BNBBEAR', 'DATA', 'SOL', 'CTSI', 'ERD', 'HBAR', 'MATIC', 'WRX', 'ZIL', 'KNC', 'REP', 'LRC', 'IQ', 'GBP', 'DGB', 'COMP', 'BKRW', 'SXP', 'SNX', 'VTHO', 'DCR', 'STORJ', 'IRIS', 'MKR', 'DAI', 'RUNE', 'MANA', 'DOGE', 'LEND', 'ZRX', 'AUD', 'FIO', 'AVA', 'IOTA', 'BAL', 'YFI', 'BLZ', 'KMD', 'JST', 'SRM', 'ANT', 'CRV', 'SAND', 'OCEAN', 'NMR', 'DOT', 'LUNA', 'IDEX', 'RSR', 'PAXG', 'WNXM', 'TRB', 'BZRX', 'SUSHI', 'YFII', 'KSM', 'EGLD', 'DIA', 'BEL', 'SWRV', 'WING', 'CREAM', 'UNI', 'AVAX', 'FLM', 'CAKE', 'XVS', 'ALPHA', 'VIDT', 'AAVE', 'NEAR', 'FIL', 'INJ', 'AERGO', 'ONE', 'AUDIO', 'CTK', 'BOT', 'KP3R', 'AXS', 'HARD', 'DNT', 'CVP', 'STRAX', 'FOR', 'UNFI', 'FRONT', 'BCHA', 'ROSE', 'SYS', 'HEGIC', 'PROM', 'SKL', 'COVER', 'GHST', 'DF', 'JUV', 'PSG', 'BTCST', 'TRU', 'DEXE', 'USDC', 'TUSD', 'PAX', 'CKB', 'TWT', 'LIT', 'SFP', 'FXS', 'DODO', 'BAKE', 'UFT', '1INCH', 'BAND', 'GRT', 'IOST', 'OMG', 'REEF', 'ACM', 'AUCTION', 'PHA', 'TVK', 'BADGER', 'FIS', 'OM', 'POND', 'DEGO', 'ALICE', 'CHZ', 'BIFI', 'LINA', 'PERP', 'RAMP', 'SUPER', 'CFX', 'XVG', 'EPS', 'AUTO', 'TKO', 'TLM', 'BTG', 'HOT', 'MIR', 'BAR', 'FORTH', 'BURGER', 'SLP', 'SHIB', 'ICP', 'AR', 'POLS', 'MDX', 'MASK', 'LPT', 'NU', 'RLC', 'CELR', 'ATM', 'ZEN', 'FTM', 'THETA', 'WIN', 'KAVA', 'XEM', 'ATA', 'GTC', 'TORN', 'COTI', 'KEEP', 'SC', 'CHR', 'STMX', 'HNT', 'FTT', 'DOCK', 'ERN', 'KLAY', 'UTK', 'IOTX', 'BOND', 'MLN', 'LTO', 'ADX', 'QUICK', 'C98', 'CLV', 'QNT', 'FLOW', 'XEC', 'MINA', 'RAY', 'FARM', 'ALPACA', 'ORN', 'MBOX', 'WAXP', 'TRIBE', 'GNO', 'MTL', 'OGN', 'POLY', 'DYDX', 'ELF', 'USDP', 'GALA', 'SUN', 'ILV', 'REN', 'YGG', 'STX', 'FET', 'ARPA', 'LSK', 'FIDA', 'DENT', 'AGLD', 'RAD', 'HIVE', 'STPT', 'BETA', 'RARE', 'TROY', 'CHESS', 'SCRT', 'CELO', 'DAR', 'BNX', 'RGT', 'LAZIO', 'OXT', 'MOVR', 'CITY', 'ENS', 'ANKR', 'QI', 'POWR', 'JASMY', 'AMP', 'PLA', 'PYR', 'RNDR', 'ALCX', 'MC', 'COCOS', 'ANY', 'BICO', 'FLUX', 'REQ', 'VOXEL', 'COS', 'CTXC', 'HIGH', 'CVX', 'PEOPLE', 'OOKI', 'MDT', 'NULS', 'SPELL', 'UST', 'JOE', 'DUSK', 'ACH', 'IMX', 'GLMR', 'UMA', 'LOKA', 'API3', 'ACA', 'ANC', 'XNO', 'WOO', 'TFUEL', 'ASTR', 'GMT', 'KDA', 'APE', 'ALPINE', 'BSW', 'SANTOS', 'MULTI', 'PORTO', 'BTTC', 'MBL', 'MOB', 'NEXO', 'GAL', 'LDO', 'EPX', 'STEEM', 'CVC', 'REI', 'DREP', 'AKRO', 'PUNDIX', 'LUNC', 'USTC', 'OP', 'OG', 'KEY', 'ASR', 'FIRO', 'NKN', 'GTO', 'WBTC', 'LEVER', 'GLM', 'SSV', 'STG', 'ARK', 'BETH', 'LOOM', 'SNM', 'AMB', 'PHB', 'GAS', 'NEBL', 'PROS', 'VIB', 'GMX', 'AGIX', 'SNT', 'POLYX', 'APT', 'QKC', 'OSMO']
groups  = ['CryptoClassicsVIP', 'CRYPTONOTESMOHSIN','whalesguide', 'FedRussianInsiders', 'TCGFORYOU', 'UltimateCryptoSignals', 'coin_signals', 'cryptopapasignals','AlphaTradeZone','godx_signals',  'cryptovipsignalta', 'Thebull_crypto','QualitySignalsChannel', 'CryptoSignalCrazy','legendofscalping','henryscalper','bestcryptotutor','cryptocrownofficial',"binancefuturesignal","cryptosignalalert","pumpsignals1","freebitmexsignals","btctradingclub", "Dash2Trade", "next10xgem1", "Wallstreetbets_trading", "BitmexCourses", "CryptoProfitCoach", "The_CryptoLogy_Signals"]

In [39]:
print(len(binance_coins))

368


In [40]:
!pip install snscrape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
import snscrape.modules.telegram as telegram
import pandas as pd

tweets_list = []
for i in groups:
    for x,tel in enumerate(telegram.TelegramChannelScraper(i).get_items()):
        date = str(tel.date).split('+')[0].split()[0]
        time = str(tel.date).split('+')[0].split()[1]
        if  (int(date.split('-')[1])<=9 and int(date.split('-')[2])<=14):
            break
        tweets_list.append([i,date, time, tel.content])

In [42]:
rdd = spark.sparkContext.parallelize(tweets_list)

In [43]:
columns=['Name', 'Date',' Time', 'Text']

In [44]:
df=rdd.toDF(columns)

In [45]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Date: string (nullable = true)
 |--  Time: string (nullable = true)
 |-- Text: string (nullable = true)



In [46]:
df.show(truncate=False)

+-----------------+----------+--------+---------------------------------------------------------------------------------------------------------------------------------------------------+
|Name             |Date      | Time   |Text                                                                                                                                               |
+-----------------+----------+--------+---------------------------------------------------------------------------------------------------------------------------------------------------+
|CryptoClassicsVIP|2022-11-24|21:27:22|#LTCUSDT  Trade Type: long position  Entry zone 46.40 - 52.50  Amount: 3% from deposit   Target: 1) 61.69 2) 70.90 3) 89.75  Stop 43.20            |
|CryptoClassicsVIP|2022-11-09|21:26:58|#BNBUSDT  Trade Type: long position  Entry zone 277 - 297  Amount: 3% from deposit   Target: 1) 330 2) 367 3) 404  Stop 256                        |
|CryptoClassicsVIP|2022-11-09|21:22:57|#ALGOUSDT  Trade Type

In [47]:
print(df.count(), len(df.columns))

20779 4


In [48]:
from pyspark.sql.functions import col,udf

In [49]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
exclude.remove('.')
exclude.remove('-')
lemma = WordNetLemmatizer()
def text_cleaning(text):
    stop_free = ' '.join([word for word in str(text).split() if word not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = ' '.join([lemma.lemmatize(word) for word in punc_free.split()])
    return normalized.split()


In [50]:
t_cleaning=udf(lambda q : text_cleaning(q))

In [51]:
from pyspark.sql.functions import lit

In [52]:
df1=df.withColumn("Cleaned Text", lit(t_cleaning(col("Text"))))

In [53]:
df1.show()

+-----------------+----------+--------+--------------------+--------------------+
|             Name|      Date|    Time|                Text|        Cleaned Text|
+-----------------+----------+--------+--------------------+--------------------+
|CryptoClassicsVIP|2022-11-24|21:27:22|#LTCUSDT  Trade T...|[LTCUSDT, Trade, ...|
|CryptoClassicsVIP|2022-11-09|21:26:58|#BNBUSDT  Trade T...|[BNBUSDT, Trade, ...|
|CryptoClassicsVIP|2022-11-09|21:22:57|#ALGOUSDT  Trade ...|[ALGOUSDT, Trade,...|
|CryptoClassicsVIP|2022-11-08|19:32:16|#BTCUSDT  Trade T...|[BTCUSDT, Trade, ...|
|CryptoClassicsVIP|2022-11-08|16:34:31|#CAKEUSDT  Trade ...|[CAKEUSDT, Trade,...|
|CryptoClassicsVIP|2022-11-05|11:35:09|#LTCUSDT  Trade T...|[LTCUSDT, Trade, ...|
|CryptoClassicsVIP|2022-11-05|00:36:46|#DYDXUSDT  Trade ...|[DYDXUSDT, Trade,...|
|CryptoClassicsVIP|2022-11-04|16:29:12|#BNBUSDT  Trade T...|[BNBUSDT, Trade, ...|
|CryptoClassicsVIP|2022-10-31|09:58:10|#BNBUSDT  Trade T...|[BNBUSDT, Trade, ...|
|CryptoClassicsV

In [81]:
import pandas as pd

In [54]:
#from google.colab import drive

#drive.mount('/content/drive')

In [55]:
#.repartition('Name')
#.sortWithinPartitions('Date')

In [83]:
# Spark df to Pandas df
df_pd = df1.toPandas()

In [84]:
df_pd[df_pd['Name']=='CryptoClassicsVIP']

,Name,Date,Time,Text,Cleaned Text
0,CryptoClassicsVIP,2022-11-24,21:27:22,#LTCUSDT Trade Type: long position Entry zon...,"[LTCUSDT, Trade, Type, long, position, Entry, ..."
1,CryptoClassicsVIP,2022-11-09,21:26:58,#BNBUSDT Trade Type: long position Entry zon...,"[BNBUSDT, Trade, Type, long, position, Entry, ..."
2,CryptoClassicsVIP,2022-11-09,21:22:57,#ALGOUSDT Trade Type: long position Entry zo...,"[ALGOUSDT, Trade, Type, long, position, Entry,..."
3,CryptoClassicsVIP,2022-11-08,19:32:16,#BTCUSDT Trade Type: long position Entry zon...,"[BTCUSDT, Trade, Type, long, position, Entry, ..."
4,CryptoClassicsVIP,2022-11-08,16:34:31,#CAKEUSDT Trade Type: long position Entry zo...,"[CAKEUSDT, Trade, Type, long, position, Entry,..."
...,...,...,...,...,...
73,CryptoClassicsVIP,2022-09-18,21:23:48,#UNIUSDT Trade Type: long position Entry zon...,"[UNIUSDT, Trade, Type, long, position, Entry, ..."
74,CryptoClassicsVIP,2022-09-18,21:08:02,#DASHUSDT Trade Type: long position Entry zo...,"[DASHUSDT, Trade, Type, long, position, Entry,..."
75,CryptoClassicsVIP,2022-09-16,14:30:31,"White House Releases Bitcoin, Crypto Regulator...","[White, House, Releases, Bitcoin, Crypto, Regu..."
76,CryptoClassicsVIP,2022-09-15,14:04:13,The Ethereum Merge is completed — link,"[The, Ethereum, Merge, completed, —, link]"


In [87]:
os.chdir("/content/content/")
# df_pd[df_pd['Name']=='CryptoClassicsVIP'].to_csv("SD.csv")
pd.to_pickle(df_pd,"hello.pckl")

In [63]:
os.chdir("/content/drive/MyDrive/pen")
if not os.path.exists('Channels'):
    os.mkdir('Channels')
os.chdir(os.path.join(os.getcwd(),'Channels'))
for i in groups:
    if not os.path.exists(i):
        os.mkdir(i)
    os.chdir(os.path.join(os.getcwd(),i))
    df_pd[df_pd['Name']==i].to_csv('1_'+i+'.csv', mode='w', index=False, header=False)
    os.chdir('/content/drive/MyDrive/pen/Channels')
os.chdir("/content/drive/MyDrive/pen")
with open('datetime.txt', 'w') as f:
    f.write(str(datetime.now()))

ImportError: ignored

In [61]:
for i in groups:
 df1.write.partitionBy('Name').format("csv").mode("overwrite").option("header","true").save("/content/drive/MyDrive/trying")
with open('/datetime.txt', 'w') as f:
  f.write(str(datetime.now()))
#   #df.repartition(df.Name==i).write.format("csv").mode("overwrite").option("header","true").save("hi")

In [ ]:
#df.load("file")
#for i in groups:
 # df.repartition(df.Name).write.format("csv").mode("overwrite").save("hello")
  #df.to_csv()
  #df.write.option("header","true").csv("/content/file")
#with open('/datetime.txt', 'w') as f:
  #f.write(str(datetime.now()))
  

In [ ]:
#df1=spark.read.format("csv").option("header","true").load("outputPath/'i'.csv")
#df1.show()